## Add our Accounts to the Database

In this code, we will create a script that allows us to add a row with user information to our cred.db

Output: We have sucessfully added one row of user information into our database! 

*Note:* re run this script to add as many users as we want to

In [ ]:
import sqlite3
from cryptography.fernet import Fernet
import os
from datetime import datetime
import pandas as pd

# read the credentials from the Excel file
df = pd.read_excel(FILE_PATH)

# Open the key file to read the encryption key
with open("../credentials/key.key", "rb") as key_file:
    key = key_file.read()

# Initialize the Fernet cipher with the key
fernet = Fernet(key)

# Define function
def add_accounts():
    # Connect to or create the SQLite DB
    conn = sqlite3.connect("../database/scraper.db")
    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    
    # This is a away to iterate through each row in the dataframe
    # itterows() is a pandas method that returns a tuple for each row (the index and the row data)
    # we unpack the tuple into _ (we ignore the index) and row (which contains the row data as a Series)
    # so we are just grabbing the row data to use in the loop
    for _, row in df.iterrows():
    
        # input info into the database
        username = row['username']
        password = row['password']
        proxy = row['proxy']
        last_used = datetime.now().isoformat() # Get current time in ISO format
        status = 'active' # Default status

        # Encrypt the password
        # password.encode() converts the password string to bytes so fernet.encrypt can process it and encrypt it
        # so now we have an encryped password in bytes, we decode it to store as a string in the database

        encrypted_password = fernet.encrypt(password.encode()).decode()

        # Insert into DB
        cursor.execute("""
            INSERT INTO accounts (username, encrypted_password, proxy, last_used, status)
            VALUES (?, ?, ?, ?, ?)
        """, (username, encrypted_password, proxy or None, last_used or None, status))

        # Commit the changes to the database
        conn.commit()
        # Close the cursor and connection
    cursor.close()
   

    print("Accounts added successfully.")

In [10]:
add_accounts()

Accounts added successfully.
